# Import of required libraries

In [1]:
import dash
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.graph_objs as go
import plotly.express as px
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Creation of dataframes

## Reading

In [45]:
covid_data = 'Data/covid-data.csv'
education_GDP_data = 'Data/Education-GDP.csv'
GDP_data = 'Data/GDP.csv'
gini_data = 'Data/gini.csv'
poverty_data = 'Data/poverty-explorer.csv'
trade_data = 'Data/trade.csv'
general_economy_data = 'Data/data_econo_covid.csv'

In [46]:
df_covid = pd.read_csv(covid_data, delimiter=',')
df_education = pd.read_csv(education_GDP_data, delimiter=',')
df_GDP = pd.read_csv(GDP_data, delimiter=',')
df_inequality = pd.read_csv(gini_data, delimiter=',')
df_poverty = pd.read_csv(poverty_data, delimiter=',') #--> I dont want to use it
df_trade = pd.read_csv(trade_data, delimiter=',')
df_economy = pd.read_csv(general_economy_data, delimiter=';')

In [5]:
df_economy

,iso_code,location,date,total_cases,total_deaths,stringency_index,population,gdp_per_capita,human_development_index,TC,TD,STI,POP,GDPCAP
0,AFG,Afghanistan,2019-12-31,0.0,0.0,0,38928341,"1803,987","0,498",#¡NUM!,#¡NUM!,#¡NUM!,"17,4772331","7,497754494"
1,AFG,Afghanistan,2020-01-01,0.0,0.0,0,38928341,"1803,987","0,498",#¡NUM!,#¡NUM!,#¡NUM!,"17,4772331","7,497754494"
2,AFG,Afghanistan,2020-01-02,0.0,0.0,0,38928341,"1803,987","0,498",#¡NUM!,#¡NUM!,#¡NUM!,"17,4772331","7,497754494"
3,AFG,Afghanistan,2020-01-03,0.0,0.0,0,38928341,"1803,987","0,498",#¡NUM!,#¡NUM!,#¡NUM!,"17,4772331","7,497754494"
4,AFG,Afghanistan,2020-01-04,0.0,0.0,0,38928341,"1803,987","0,498",#¡NUM!,#¡NUM!,#¡NUM!,"17,4772331","7,497754494"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50413,ZWE,Zimbabwe,2020-10-15,8055.0,231.0,"76,85",14862927,"1899,775","0,535","8,994048296","5,442417711","4,34185547","16,51438055","7,549490737"
50414,ZWE,Zimbabwe,2020-10-16,8075.0,231.0,"76,85",14862927,"1899,775","0,535","8,996528148","5,442417711","4,34185547","16,51438055","7,549490737"
50415,ZWE,Zimbabwe,2020-10-17,8099.0,231.0,"76,85",14862927,"1899,775","0,535","8,999495876","5,442417711","4,34185547","16,51438055","7,549490737"
50416,ZWE,Zimbabwe,2020-10-18,8110.0,231.0,"76,85",14862927,"1899,775","0,535","9,000853147","5,442417711","4,34185547","16,51438055","7,549490737"


## Filtering by area

In [47]:
# We don't consider the islands

south_america = ['Chile', 'Argentina','Uruguay','Paraguay','Bolivia','Peru','Brazil','Ecuador',
                'Venezuela', 'Colombia','Guyana','Suriname','French Guiana']
north_america = ['Canada','United States', 'Mexico','Guatemala','Honduras','Nicaragua','Costa Rica','Panama',
                'Dominican Republic', 'Jamaica','Cuba','Bahamas','Belize']

In [48]:
americas = south_america + north_america
df_americas = df_GDP[df_GDP['Entity'].isin(americas)]
df_covid = df_covid[df_covid['location'].isin(americas)]

In [49]:
# df_poverty_renamed = df_poverty.rename(columns={'Country': 'Entity'})
df_americas = pd.merge(df_americas, df_education, on=['Entity', 'Year'], how='left')
df_americas = pd.merge(df_americas, df_inequality, on=['Entity', 'Year'], how='left')
df_americas = pd.merge(df_americas, df_trade, on=['Entity', 'Year'], how='left')
# df_americas = pd.merge(df_americas, df_poverty_renamed, on=['Entity', 'Year'], how='left')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'Code_x'} in the result is deprecated and will raise a MergeError in a future version.



### Modification of format dataframe df_economy

In [50]:
df_economy_renamed = df_economy.rename(columns={'location': 'Entity'})
df_economy_renamed['date'] = pd.to_datetime(df_economy_renamed['date'])
df_economy_renamed['Year'] = df_economy_renamed['date'].dt.year
df_economy_renamed['Month'] = df_economy_renamed['date'].dt.month

# df_economy_grouped = df_economy_renamed.groupby(['Year', 'iso_code']).agg({'columna_a_agregar': 'sum'}).reset_index()
df_economy_americas = df_economy_renamed[df_economy_renamed['Entity'].isin(americas)]

### Modification of format dataframe df_covid

In [51]:
df_covid['date'] = pd.to_datetime(df_covid['date'])
df_covid['Year'] = df_covid['date'].dt.year
df_covid['Month'] = df_covid['date'].dt.month

# Analysis df_americas

In [52]:
df_americas.drop(['Code_x'], axis=1, inplace=True)
df_americas.drop(['Code_y'], axis=1, inplace=True)



In [53]:
description = df_americas.describe()
description

,Year,"Gross domestic product, constant prices - Percent change - Observations","Gross domestic product, constant prices - Percent change - Forecasts",Historical and more recent expenditure estimates,Gini coefficient,Trade (% of GDP)
count,1172.000000,1025.000000,171.000000,566.000000,446.000000,947.000000
mean,2003.923208,2.795708,3.741924,4.369025,0.478737,65.040680
std,14.110794,4.765327,7.596325,1.517905,0.074141,36.062448
min,1980.000000,-23.508000,-29.995000,0.773950,0.310191,11.545672
25%,1992.000000,0.852000,2.007500,3.283885,0.431895,39.829510
50%,2004.000000,3.394000,2.984000,4.350055,0.486623,57.777030
75%,2016.000000,5.099000,3.800000,5.360971,0.533703,83.175322
max,2028.000000,43.480000,62.288000,8.561060,0.632951,274.973100


In [11]:
df_americas.columns


Index(['Entity', 'Year',
       'Gross domestic product, constant prices - Percent change - Observations',
       'Gross domestic product, constant prices - Percent change - Forecasts',
       'Historical and more recent expenditure estimates', 'Gini coefficient',
       'Trade (% of GDP)'],
      dtype='object')

In [54]:
duplicate_values = df_americas.duplicated().any()
print(f'DataFrame has duplicate rows?: {duplicate_values}')

DataFrame has duplicate rows?: False


In [55]:
percentage_null = df_americas.isnull().mean() * 100
print(f" The percentage of null values for each column is  :\n {percentage_null}")

 The percentage of null values for each column is  :
 Entity                                                                      0.000000
Year                                                                        0.000000
Gross domestic product, constant prices - Percent change - Observations    12.542662
Gross domestic product, constant prices - Percent change - Forecasts       85.409556
Historical and more recent expenditure estimates                           51.706485
Gini coefficient                                                           61.945392
Trade (% of GDP)                                                           19.197952
dtype: float64


In [56]:
percentage_null_per_year = df_americas.groupby('Year').apply(lambda x: x.isnull().mean() * 100)
print(percentage_null_per_year)

      Entity  Year  \
Year                 
1980     0.0   0.0   
1981     0.0   0.0   
1982     0.0   0.0   
1983     0.0   0.0   
1984     0.0   0.0   
1985     0.0   0.0   
1986     0.0   0.0   
1987     0.0   0.0   
1988     0.0   0.0   
1989     0.0   0.0   
1990     0.0   0.0   
1991     0.0   0.0   
1992     0.0   0.0   
1993     0.0   0.0   
1994     0.0   0.0   
1995     0.0   0.0   
1996     0.0   0.0   
1997     0.0   0.0   
1998     0.0   0.0   
1999     0.0   0.0   
2000     0.0   0.0   
2001     0.0   0.0   
2002     0.0   0.0   
2003     0.0   0.0   
2004     0.0   0.0   
2005     0.0   0.0   
2006     0.0   0.0   
2007     0.0   0.0   
2008     0.0   0.0   
2009     0.0   0.0   
2010     0.0   0.0   
2011     0.0   0.0   
2012     0.0   0.0   
2013     0.0   0.0   
2014     0.0   0.0   
2015     0.0   0.0   
2016     0.0   0.0   
2017     0.0   0.0   
2018     0.0   0.0   
2019     0.0   0.0   
2020     0.0   0.0   
2021     0.0   0.0   
2022     0.0   0.0   
2023     0

In [15]:
fig = px.line(df_americas, x='Year', y='Gini coefficient', color='Entity', title='Evolution  of Gini Coefficient')
fig.show()


# Analysis df_covid

In [16]:
df_covid.dtypes

iso_code                                           object
continent                                          object
location                                           object
date                                       datetime64[ns]
total_cases                                       float64
                                                ...      
excess_mortality_cumulative                       float64
excess_mortality                                  float64
excess_mortality_cumulative_per_million           float64
Year                                                int64
Month                                               int64
Length: 69, dtype: object

In [17]:
df_covid.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

In [18]:
df_covid

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million,Year,Month
13716,ARG,South America,Argentina,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,76.67,0.845,45510324.0,NaN,NaN,NaN,NaN,2020,1
13717,ARG,South America,Argentina,2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,76.67,0.845,45510324.0,NaN,NaN,NaN,NaN,2020,1
13718,ARG,South America,Argentina,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,5.0,76.67,0.845,45510324.0,NaN,NaN,NaN,NaN,2020,1
13719,ARG,South America,Argentina,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,5.0,76.67,0.845,45510324.0,NaN,NaN,NaN,NaN,2020,1
13720,ARG,South America,Argentina,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,5.0,76.67,0.845,45510324.0,NaN,NaN,NaN,NaN,2020,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374120,VEN,South America,Venezuela,2024-03-01,552695.0,0.0,0.0,5856.0,0.0,0.0,...,0.8,72.06,0.711,28301700.0,NaN,NaN,NaN,NaN,2024,3
374121,VEN,South America,Venezuela,2024-03-02,552695.0,0.0,0.0,5856.0,0.0,0.0,...,0.8,72.06,0.711,28301700.0,NaN,NaN,NaN,NaN,2024,3
374122,VEN,South America,Venezuela,2024-03-03,552695.0,0.0,0.0,5856.0,0.0,0.0,...,0.8,72.06,0.711,28301700.0,NaN,NaN,NaN,NaN,2024,3
374123,VEN,South America,Venezuela,2024-03-04,552695.0,0.0,0.0,5856.0,0.0,0.0,...,0.8,72.06,0.711,28301700.0,NaN,NaN,NaN,NaN,2024,3


In [57]:
variance_threshold = 0.1  # Ejemplo
df_var = df_covid.var()
columnas_baja_var = df_var[df_var < variance_threshold].index
df_covid = df_covid.drop(columnas_baja_var, axis=1)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [58]:
threshold = 0.5 
df_covid = df_covid.dropna(thresh=int(threshold * len(df_covid)), axis=1)

In [59]:
df_covid.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,population,Year,Month
13716,ARG,South America,Argentina,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,...,191.032,5.5,16.2,27.7,NaN,5.0,76.67,45510324.0,2020,1
13717,ARG,South America,Argentina,2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,...,191.032,5.5,16.2,27.7,NaN,5.0,76.67,45510324.0,2020,1
13718,ARG,South America,Argentina,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,191.032,5.5,16.2,27.7,NaN,5.0,76.67,45510324.0,2020,1
13719,ARG,South America,Argentina,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,191.032,5.5,16.2,27.7,NaN,5.0,76.67,45510324.0,2020,1
13720,ARG,South America,Argentina,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,191.032,5.5,16.2,27.7,NaN,5.0,76.67,45510324.0,2020,1


In [60]:
columns_to_remove = ['continent','new_cases_smoothed','new_deaths_smoothed','total_cases_per_million',
                     'new_cases_per_million', 'new_cases_smoothed_per_million',
                     'total_deaths_per_million', 'new_deaths_per_million',
                     'new_deaths_smoothed_per_million','aged_65_older', 'aged_70_older','cardiovasc_death_rate', 'diabetes_prevalence',
                     'female_smokers', 'male_smokers', 'hospital_beds_per_thousand','life_expectancy', 'handwashing_facilities',
                     'population','median_age','population_density','new_cases','new_deaths','new_vaccinations_smoothed',
                     'new_vaccinations_smoothed_per_million','new_people_vaccinated_smoothed'
                     ]

df_covid = df_covid.drop(columns = columns_to_remove)

In [61]:
df_covid.columns

Index(['iso_code', 'location', 'date', 'total_cases', 'total_deaths',
       'stringency_index', 'gdp_per_capita', 'extreme_poverty', 'Year',
       'Month'],
      dtype='object')

# Analysis df_economy_americas

In [62]:
df_economy_americas


,iso_code,Entity,date,total_cases,total_deaths,stringency_index,population,gdp_per_capita,human_development_index,TC,TD,STI,POP,GDPCAP,Year,Month
1672,ARG,Argentina,2020-02-11,NaN,NaN,"11,11",45195777,"18933,907","0,825",#¡NUM!,#¡NUM!,"2,407845604","17,62651421","9,848709615",2020,2
1673,ARG,Argentina,2020-02-12,NaN,NaN,"11,11",45195777,"18933,907","0,825",#¡NUM!,#¡NUM!,"2,407845604","17,62651421","9,848709615",2020,2
1674,ARG,Argentina,2020-02-13,NaN,NaN,"11,11",45195777,"18933,907","0,825",#¡NUM!,#¡NUM!,"2,407845604","17,62651421","9,848709615",2020,2
1675,ARG,Argentina,2020-02-14,NaN,NaN,"11,11",45195777,"18933,907","0,825",#¡NUM!,#¡NUM!,"2,407845604","17,62651421","9,848709615",2020,2
1676,ARG,Argentina,2020-02-15,NaN,NaN,"11,11",45195777,"18933,907","0,825",#¡NUM!,#¡NUM!,"2,407845604","17,62651421","9,848709615",2020,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49498,VEN,Venezuela,2020-10-15,85005.0,714.0,"87,96",28435943,"16745,022","0,761","11,35046536","6,570882962","4,476882166","17,1631645","9,725856299",2020,10
49499,VEN,Venezuela,2020-10-16,85469.0,720.0,NaN,28435943,"16745,022","0,761","11,35590902","6,579251212",#¡NUM!,"17,1631645","9,725856299",2020,10
49500,VEN,Venezuela,2020-10-17,85469.0,720.0,NaN,28435943,"16745,022","0,761","11,35590902","6,579251212",#¡NUM!,"17,1631645","9,725856299",2020,10
49501,VEN,Venezuela,2020-10-18,85758.0,725.0,NaN,28435943,"16745,022","0,761","11,35928466","6,586171655",#¡NUM!,"17,1631645","9,725856299",2020,10


In [63]:
type_of_data = df_economy_renamed.dtypes
type_of_data

iso_code                           object
Entity                             object
date                       datetime64[ns]
total_cases                       float64
total_deaths                      float64
stringency_index                   object
population                          int64
gdp_per_capita                     object
human_development_index            object
TC                                 object
TD                                 object
STI                                object
POP                                object
GDPCAP                             object
Year                                int64
Month                               int64
dtype: object

In [64]:
drop = ['TC', 'TD', 'STI', 'POP', 'GDPCAP','population']
df_economy_americas = df_economy_americas.drop(columns = drop)

In [65]:
df_economy_americas

,iso_code,Entity,date,total_cases,total_deaths,stringency_index,gdp_per_capita,human_development_index,Year,Month
1672,ARG,Argentina,2020-02-11,NaN,NaN,"11,11","18933,907","0,825",2020,2
1673,ARG,Argentina,2020-02-12,NaN,NaN,"11,11","18933,907","0,825",2020,2
1674,ARG,Argentina,2020-02-13,NaN,NaN,"11,11","18933,907","0,825",2020,2
1675,ARG,Argentina,2020-02-14,NaN,NaN,"11,11","18933,907","0,825",2020,2
1676,ARG,Argentina,2020-02-15,NaN,NaN,"11,11","18933,907","0,825",2020,2
...,...,...,...,...,...,...,...,...,...,...
49498,VEN,Venezuela,2020-10-15,85005.0,714.0,"87,96","16745,022","0,761",2020,10
49499,VEN,Venezuela,2020-10-16,85469.0,720.0,NaN,"16745,022","0,761",2020,10
49500,VEN,Venezuela,2020-10-17,85469.0,720.0,NaN,"16745,022","0,761",2020,10
49501,VEN,Venezuela,2020-10-18,85758.0,725.0,NaN,"16745,022","0,761",2020,10


In [66]:
columnas_a_convertir = ['stringency_index', 'human_development_index','gdp_per_capita']

# Convertir las columnas especificadas a float
for columna in columnas_a_convertir:
    df_economy_americas[columna] = df_economy_americas[columna].str.replace(',', '.').astype(float)


In [67]:
df_economy_americas

,iso_code,Entity,date,total_cases,total_deaths,stringency_index,gdp_per_capita,human_development_index,Year,Month
1672,ARG,Argentina,2020-02-11,NaN,NaN,11.11,18933.907,0.825,2020,2
1673,ARG,Argentina,2020-02-12,NaN,NaN,11.11,18933.907,0.825,2020,2
1674,ARG,Argentina,2020-02-13,NaN,NaN,11.11,18933.907,0.825,2020,2
1675,ARG,Argentina,2020-02-14,NaN,NaN,11.11,18933.907,0.825,2020,2
1676,ARG,Argentina,2020-02-15,NaN,NaN,11.11,18933.907,0.825,2020,2
...,...,...,...,...,...,...,...,...,...,...
49498,VEN,Venezuela,2020-10-15,85005.0,714.0,87.96,16745.022,0.761,2020,10
49499,VEN,Venezuela,2020-10-16,85469.0,720.0,NaN,16745.022,0.761,2020,10
49500,VEN,Venezuela,2020-10-17,85469.0,720.0,NaN,16745.022,0.761,2020,10
49501,VEN,Venezuela,2020-10-18,85758.0,725.0,NaN,16745.022,0.761,2020,10


# Input Data

## df_americas

In [68]:
df_americas.columns

Index(['Entity', 'Year',
       'Gross domestic product, constant prices - Percent change - Observations',
       'Gross domestic product, constant prices - Percent change - Forecasts',
       'Historical and more recent expenditure estimates', 'Gini coefficient',
       'Trade (% of GDP)'],
      dtype='object')

In [70]:
nul_values = df_americas.isnull().sum()
porcentage_nul = (nul_values / len(df_americas)) * 100
print(porcentage_nul)

Entity                                                                      0.000000
Year                                                                        0.000000
Gross domestic product, constant prices - Percent change - Observations    12.542662
Gross domestic product, constant prices - Percent change - Forecasts       85.409556
Historical and more recent expenditure estimates                           51.706485
Gini coefficient                                                           61.945392
Trade (% of GDP)                                                           19.197952
dtype: float64


In [71]:
df_americas2 = df_americas.copy()

In [72]:
from sklearn.impute import KNNImputer

columnas_para_imputar = [
    'Gross domestic product, constant prices - Percent change - Observations',
    'Historical and more recent expenditure estimates',
    'Gini coefficient',
    'Trade (% of GDP)'
]
datos_para_imputar = df_americas2[columnas_para_imputar]
knn_imputer = KNNImputer(n_neighbors=5)
df_americas2_knn_imputed = knn_imputer.fit_transform(datos_para_imputar)
df_americas2_knn_imputed_df = pd.DataFrame(df_americas2_knn_imputed, columns=columnas_para_imputar)

df_americas2[columnas_para_imputar] = df_americas2_knn_imputed

In [73]:
nul_values = df_americas2.isnull().sum()
porcentage_nul = (nul_values / len(df_americas2)) * 100
print(porcentage_nul)

Entity                                                                      0.000000
Year                                                                        0.000000
Gross domestic product, constant prices - Percent change - Observations     0.000000
Gross domestic product, constant prices - Percent change - Forecasts       85.409556
Historical and more recent expenditure estimates                            0.000000
Gini coefficient                                                            0.000000
Trade (% of GDP)                                                            0.000000
dtype: float64


## df_economy_americas

In [74]:
nul_values = df_economy_americas.isnull().sum()
porcentage_nul = (nul_values / len(df_economy_americas)) * 100
print(porcentage_nul)

iso_code                    0.000000
Entity                      0.000000
date                        0.000000
total_cases                 5.475361
total_deaths               11.597810
stringency_index            6.387921
gdp_per_capita              3.633649
human_development_index     0.000000
Year                        0.000000
Month                       0.000000
dtype: float64


In [75]:
df_economy_americas.dtypes

iso_code                           object
Entity                             object
date                       datetime64[ns]
total_cases                       float64
total_deaths                      float64
stringency_index                  float64
gdp_per_capita                    float64
human_development_index           float64
Year                                int64
Month                               int64
dtype: object

In [76]:
df_economy_americas2 = df_economy_americas.copy()

columnas_para_imputar = [
    'total_cases',
    'total_deaths',
    'stringency_index',
    'gdp_per_capita'
]
imputador = KNNImputer(n_neighbors=5)
datos_para_imputar = df_economy_americas2[columnas_para_imputar]
df_economy_americas2_imputed = imputador.fit_transform(datos_para_imputar)
df_economy_americas2_imputed_df = pd.DataFrame(df_economy_americas2_imputed, columns=columnas_para_imputar)

df_economy_americas2[columnas_para_imputar] = df_economy_americas2_imputed

In [77]:
nul_values = df_economy_americas2.isnull().sum()
porcentage_nul = (nul_values / len(df_economy_americas2)) * 100
print(porcentage_nul)

iso_code                   0.0
Entity                     0.0
date                       0.0
total_cases                0.0
total_deaths               0.0
stringency_index           0.0
gdp_per_capita             0.0
human_development_index    0.0
Year                       0.0
Month                      0.0
dtype: float64


# Prediction Data

## df_economy_americas2

In [79]:
nulos_per_year = df_economy_americas2.groupby('Year').apply(lambda x: x.isnull().sum())
print(nulos_per_year)

      iso_code  Entity  date  total_cases  total_deaths  stringency_index  \
Year                                                                        
2019         0       0     0            0             0                 0   
2020         0       0     0            0             0                 0   

      gdp_per_capita  human_development_index  Year  Month  
Year                                                        
2019               0                        0     0      0  
2020               0                        0     0      0  


In [81]:
df_economy_americas2

,iso_code,Entity,date,total_cases,total_deaths,stringency_index,gdp_per_capita,human_development_index,Year,Month
1672,ARG,Argentina,2020-02-11,950657.8,25365.0,11.110,18933.907,0.825,2020,2
1673,ARG,Argentina,2020-02-12,950657.8,25365.0,11.110,18933.907,0.825,2020,2
1674,ARG,Argentina,2020-02-13,950657.8,25365.0,11.110,18933.907,0.825,2020,2
1675,ARG,Argentina,2020-02-14,950657.8,25365.0,11.110,18933.907,0.825,2020,2
1676,ARG,Argentina,2020-02-15,950657.8,25365.0,11.110,18933.907,0.825,2020,2
...,...,...,...,...,...,...,...,...,...,...
49498,VEN,Venezuela,2020-10-15,85005.0,714.0,87.960,16745.022,0.761,2020,10
49499,VEN,Venezuela,2020-10-16,85469.0,720.0,32.036,16745.022,0.761,2020,10
49500,VEN,Venezuela,2020-10-17,85469.0,720.0,32.036,16745.022,0.761,2020,10
49501,VEN,Venezuela,2020-10-18,85758.0,725.0,32.036,16745.022,0.761,2020,10


# Dash API

In [25]:
df_americas.columns

Index(['Entity', 'Year',
       'Gross domestic product, constant prices - Percent change - Observations',
       'Gross domestic product, constant prices - Percent change - Forecasts',
       'Historical and more recent expenditure estimates', 'Gini coefficient',
       'Trade (% of GDP)'],
      dtype='object')

In [82]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

# Map
fig_map = px.scatter_geo(df_americas2, locations="Entity", locationmode='country names',
                         hover_name="Entity", projection="natural earth")


# Navbar
navbar = dbc.NavbarSimple(
    children=[
        dbc.NavItem(dbc.NavLink("Economic Factor", href="/page-1")),
        dbc.NavItem(dbc.NavLink("Covid Factor", href="/page-2")),
    ],
    brand="My first DashBoard",
    brand_href="/",
    color="primary",
    dark=True,
)

# Layout page 1
layout_page_1 = dbc.Container([
    html.H1("Economic and Social Evolution by Country"),
    dbc.Row(dbc.Col(html.Div("Select a country by clicking on the map below:"))),
    dbc.Row(dbc.Col(dcc.Graph(id='world-map', figure=fig_map))),
    html.Div(id='country-selected'),
    dbc.Row(dbc.Col(html.H4("Choose a variable to display:"))),
    dbc.Row(dbc.Col(dcc.Dropdown(
        id='variable-dropdown',
        options=[{'label': i, 'value': i} for i in df_americas2.columns[2:]],
        value=df_americas2.columns[2]  # default value
    ))),
    dbc.Row(dbc.Col(html.Div("Graphical representation:"))),
    dbc.Row([
        dbc.Col(dcc.Graph(id='feature-graph'), width=6),
        dbc.Col([
            html.H4("Monthly Data Visualization"),
            dcc.Dropdown(
                id='monthly-data-dropdown',
                options=[{'label': i, 'value': i} for i in df_economy_americas2.columns[3:8]],  # Asume que la columna 3 tiene datos mensuales
                value=df_economy_americas2.columns[3]  # default value
            ),
            dcc.Graph(id='monthly-data-graph')  # Gráfico para datos mensuales
        ], width=6)
    ]),
    dbc.Row(dbc.Col(html.Div(id='analysis-box')))
], fluid=True)


# Layout page 2
layout_page_2 = dbc.Container([
    html.H1("COVID-19 Data Visualization"),
    dbc.Row([
        dbc.Col(html.Div("Select a country:"), width=3),
        dbc.Col(dcc.Dropdown(
            id='country-dropdown-page2',
            options=[{'label': country, 'value': country} for country in df_covid['location'].unique()],
            value='Argentina'
        ), width=9),
    ]),
    dbc.Row([
        dbc.Col(html.Div("Select a variable:"), width=3),
        dbc.Col(dcc.Dropdown(
            id='variable-dropdown-page2',
            options=[
                {'label': 'Total Cases', 'value': 'total_cases'},
                {'label': 'Total Deaths', 'value': 'total_deaths'},
                # Puedes agregar más variables aquí según sea necesario
            ],
            value='total_cases'  # Valor predeterminado
        ), width=9),
    ]),
    dbc.Row(dbc.Col(dcc.Graph(id='evolution-graph-page2'))),
], fluid=True)


# Principal page (default)
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    navbar,
    html.Div(id='page-content')
])

# Callback 1
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/page-1':
        return layout_page_1
    elif pathname == '/page-2':
        return layout_page_2
    else:
        return "Welcome! Please choose a page."

# Callback 2
@app.callback(
    [Output('country-selected', 'children'),
     Output('feature-graph', 'figure'),
     Output('monthly-data-graph', 'figure'),  # Añadido para actualizar el gráfico mensual
     Output('analysis-box', 'children')],
    [Input('world-map', 'clickData'),
     Input('variable-dropdown', 'value'),
     Input('monthly-data-dropdown', 'value')])  # Añadido para escuchar cambios en el dropdown mensual
def update_graph(clickData, selected_variable, selected_monthly_variable):
    if clickData is None:
        analysis_text = 'Choose a country and a variable to see the analysis.'
        return 'Choose a country', {}, {}, analysis_text
    else:
        country_name = clickData['points'][0]['location']
        df_filtered = df_americas2[df_americas2['Entity'] == country_name]
        df_filtered_monthly = df_economy_americas2[df_economy_americas2['Entity'] == country_name]

        fig = px.line(df_filtered, x='Year', y=selected_variable, title=f'Annual {selected_variable} for {country_name}',
                      template="plotly_white")
        fig.update_layout(
            title={'text': f'Annual {selected_variable} for {country_name}', 'x': 0.5, 'xanchor': 'center'},
            xaxis_title="Year",
            yaxis_title=selected_variable,
            height=400,  # Ajusta el tamaño del gráfico
            font=dict(size=12)  # Ajusta el tamaño del texto global, incluyendo los títulos de los ejes
        )

        fig_monthly = px.line(df_filtered_monthly, x='Month', y=selected_monthly_variable, color='Year',
                              title=f'Monthly {selected_monthly_variable} in {country_name}', template="plotly_white")
        fig_monthly.update_layout(
            title={'text': f'Monthly {selected_monthly_variable} in {country_name}', 'x': 0.5, 'xanchor': 'center'},
            xaxis_title="Month",
            yaxis_title=selected_monthly_variable,
            height=400,  # Ajusta el tamaño del gráfico para que coincida con el otro gráfico
            font=dict(size=12)  # Ajusta el tamaño del texto global, incluyendo los títulos de los ejes
        )

        analysis_text = f"Analysis for {country_name}: Here you can provide insights into the {selected_variable} trends over time..."
        
        return f'Country: {country_name}', fig, fig_monthly, analysis_text


@app.callback(
    Output('evolution-graph-page2', 'figure'),
    [Input('country-dropdown-page2', 'value'),
     Input('variable-dropdown-page2', 'value'),
    ]
)
def update_graph_page2(selected_country, selected_variable):
    filtered_df = df_covid[df_covid['location'] == selected_country]
    
    
    fig = px.bar(filtered_df, x='Month', y=selected_variable, color='Year',
                    title=f"Evolución de {selected_variable} en {selected_country} por Mes y Año")
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
